In [12]:
import os
import json
import PyPDF2
import boto3
import nltk
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from tensorflow import keras
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

In [13]:
# %% [markdown]
# ## Paso 2: Configuración de Qdrant, Amazon Bedrock y variable de modelos
# Ajusta el host/puerto de Qdrant y el ID del modelo de Amazon Bedrock según tu configuración.

# Inicializamos el cliente de Qdrant
qdrant = QdrantClient(host="localhost", port=6333)  # Cambia estos parámetros según tu entorno
collection_name = "documentos"

# Variable de modelos (ejemplo); se debe configurar con el id del modelo en Amazon Bedrock
modelos = {
    "Titan Embeddings G1 - Text" : "amazon.titan-embed-text-v1",
    "Titan Text G1 - Lite": "amazon.titan-text-lite-v1",
    "Titan Text G1 - Express": "amazon.titan-text-express-v1",
    "": "",
    "Rerank_1.0": "amazon.rerank-v1:0",
    "Claude_3.5_Sonnet" : "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "Claude_3_Sonnet": "anthropic.claude-3-sonnet-20240229-v1:0",
    "Claude_3_Haiku" : "anthropic.claude-3-5-haiku-20241022-v1:0",
    "Claude_2.1" : "anthropic.claude-v2:1",
    "Claude_Instant" : "anthropic.claude-instant-v1",
    "Claude": "anthropic.claude-v2:0"
}

# Inicializamos el cliente de Amazon Bedrock
bedrock = boto3.client('bedrock-runtime')

In [14]:
# %% [markdown]
# ## Paso 3: Función para leer el PDF y extraer el texto

def read_pdf(file_path):
    """Lee un PDF y extrae todo su texto."""
    text = ""
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# Ruta al documento
pdf_path = "documento.pdf"
print("Leyendo el PDF...")
document_text = read_pdf(pdf_path)
print("PDF leído correctamente.")

Leyendo el PDF...
PDF leído correctamente.


In [15]:
# %% [markdown]
# ## Paso 4: Función para dividir el texto en chunks
# Se define un tamaño de chunk y un solapamiento para conservar contexto entre ellos.

def chunk_text(text, chunk_size=1000, overlap=100):
    """
    Divide el texto en chunks de `chunk_size` caracteres con un solapamiento de `overlap` caracteres.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += (chunk_size - overlap)
    return chunks

print("Dividiendo el documento en chunks...")
chunks = chunk_text(document_text, chunk_size=1000, overlap=100)
print(f"Total de chunks generados: {len(chunks)}")

Dividiendo el documento en chunks...
Total de chunks generados: 478


In [16]:
# %% [markdown]
# ## Paso 5: Calcular embeddings y subir a Qdrant
# Se usa un modelo de SentenceTransformer para obtener los embeddings de cada chunk.
# Si la colección en Qdrant no existe, se crea con la dimensión correcta.

# Inicializamos el modelo de embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Obtenemos la dimensión del embedding
dimension = embedder.get_sentence_embedding_dimension()

# Creamos la colección en Qdrant si no existe
collections = qdrant.get_collections().collections
if not any(col.name == collection_name for col in collections):
    print("Creando colección en Qdrant...")
    qdrant.create_collection(
         collection_name=collection_name,
         vectors_config=VectorParams(size=dimension, distance="Cosine")
    )
    print("Colección creada.")

print("Computando embeddings de los chunks...")
embeddings = embedder.encode(chunks, show_progress_bar=True)

# Preparamos los puntos (cada punto asocia un chunk y su embedding)
points = []
for i, (chunk, vector) in enumerate(zip(chunks, embeddings)):
    points.append(PointStruct(
         id=i,
         vector=vector.tolist(),
         payload={"text": chunk}
    ))

print("Subiendo los embeddings a Qdrant...")
qdrant.upsert(collection_name=collection_name, points=points)
print("Embeddings subidos correctamente.")

Computando embeddings de los chunks...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Subiendo los embeddings a Qdrant...
Embeddings subidos correctamente.


In [17]:
# %% [markdown]
# ## Paso 6: Función para recuperar chunks relevantes desde Qdrant
# Dada una query, se calcula su embedding y se buscan los chunks más similares.

def get_relevant_chunks(query, top_k=5):
    """
    Dada una query, recupera los `top_k` chunks más relevantes desde Qdrant.
    """
    query_embedding = embedder.encode(query)
    search_result = qdrant.search(
         collection_name=collection_name,
         query_vector=query_embedding.tolist(),
         limit=top_k
    )
    # Extraemos el texto de cada chunk recuperado
    contexts = [res.payload.get("text", "") for res in search_result]
    return contexts

In [18]:
def ask_question(query, modelo, top_k=5):
    # Recupera los chunks relevantes y construye el contexto
    contexts = get_relevant_chunks(query, top_k=top_k)
    context_text = "\n".join(contexts)
    
    # Define el mensaje de sistema que contiene instrucciones y el contexto.
    # En este ejemplo se indica que se debe responder en pocas palabras.
    system_message = f"Answer the question based on the following context shortly, in at most 5 words:\n{context_text}"
    
    # Define el mensaje del usuario con la consulta.
    messages = [{"role": "user", "content": query}]
    
    # Construye el payload utilizando la estructura que funcionó en tu prueba.
    payload = {
        "anthropic_version": "bedrock-2023-05-31",  
        "system": system_message,
        "messages": messages,
        "max_tokens": 100,
        "temperature": 0.8,
        "top_p": 0.8,
    }
    
    # Convierte el payload a JSON
    payload_json = json.dumps(payload)
    
    # Selecciona el id del modelo a partir del diccionario 'modelos'
    model_id = modelos.get(modelo)
    
    # Invoca el modelo en Amazon Bedrock
    response = bedrock.invoke_model(
         modelId=model_id,
         contentType="application/json",
         body=payload_json
    )
    
    # Procesa la respuesta (para modelos Anthropic la respuesta suele estar en la clave "content")
    response_body = json.loads(response["body"].read().decode("utf-8"))
    return response_body.get("content", response_body)


In [19]:
def extract_answer_text(generated_answer):
    """
    Given the generated answer, which may be a list of dicts or a string,
    extract and return the plain text.
    """
    if isinstance(generated_answer, list) and len(generated_answer) > 0:
        # Assume the answer is in the first element under the "text" key
        return generated_answer[0].get("text", "").strip()
    elif isinstance(generated_answer, str):
        return generated_answer.strip()
    else:
        return str(generated_answer).strip()

# Modified evaluation function to include the expected answer (for later comparison)
def evaluate_qa_pairs(qa_pairs, model_name="Claude_3.5_Sonnet"):
    results = []
    for qa in qa_pairs:
        question = qa["Q"]
        expected_answer = qa["A"]
        print(f"Evaluando pregunta: {question}")
        
        # Get the generated answer from RAG
        generated_answer = ask_question(question, model_name)
        answer_text = extract_answer_text(generated_answer)
        
        result = {
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": answer_text
        }
        results.append(result)
    return results

In [20]:
with open("Expert-questions.json", "r", encoding="utf-8") as f:
    expert_qa = json.load(f)

with open("Not-expert-questions.json", "r", encoding="utf-8") as f:
    non_expert_qa = json.load(f)

# Evaluar ambos conjuntos de preguntas
non_expert_results = evaluate_qa_pairs(non_expert_qa, model_name="Claude_3.5_Sonnet")
expert_results = evaluate_qa_pairs(expert_qa, model_name="Claude_3.5_Sonnet")

Evaluando pregunta: ¿Qué son los riesgos ESG?


C:\Users\Alonso\AppData\Local\Temp\ipykernel_15372\1406710426.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant.search(


Evaluando pregunta: ¿Cuál es el objetivo principal de estas directrices ESG?
Evaluando pregunta: ¿Por qué es importante que las instituciones gestionen los riesgos ESG?
Evaluando pregunta: ¿Qué se entiende por riesgo ambiental en este contexto?
Evaluando pregunta: ¿Qué significa “riesgo de transición”?
Evaluando pregunta: ¿Qué son los riesgos físicos en el ámbito ESG?
Evaluando pregunta: ¿Qué papel tiene el EBA en la gestión de riesgos ESG?
Evaluando pregunta: ¿Cómo deben integrar las instituciones los riesgos ESG en su gestión?
Evaluando pregunta: ¿Qué es una evaluación de materialidad en riesgos ESG?
Evaluando pregunta: ¿Qué debe incluir un plan de transición ESG?
Evaluando pregunta: ¿Qué horizonte temporal se considera para los riesgos ESG?
Evaluando pregunta: ¿Qué herramientas se pueden utilizar para medir los riesgos ESG?
Evaluando pregunta: ¿Qué es un análisis de escenarios en este contexto?
Evaluando pregunta: ¿Cómo se relacionan los riesgos ESG con la sostenibilidad?
Evaluando 

In [21]:
# Imprimir las respuestas obtenidas
# print("\nResultados para preguntas no expertas:")
# for res in non_expert_results:
#     print(f"Pregunta: {res['question']}\nRespuesta: {res['answer']}\n")
# 
# print("\nResultados para preguntas expertas:")
# for res in expert_results:
#     print(f"Pregunta: {res['question']}\nRespuesta: {res['answer']}\n")

In [22]:
# Save the results to separate JSON files
with open("non_expert_results.json", "w", encoding="utf-8") as f:
    json.dump(non_expert_results, f, ensure_ascii=False, indent=4)

with open("expert_results.json", "w", encoding="utf-8") as f:
    json.dump(expert_results, f, ensure_ascii=False, indent=4)